In [2]:
import cv2
import numpy as np

mnist = np.load("../Dataset/MNIST/mnist.npz")
x_train, y_train = mnist["x_train"], mnist["y_train"]
x_test, y_test = mnist["x_test"], mnist["y_test"]
index = 1
for image, label in zip(x_train, y_train):
    cv2.imwrite(f"../Dataset/MNIST/images/{index:05d}_{label}.jpg", image)
    index += 1
for image, label in zip(x_test, y_test):
    cv2.imwrite(f"../Dataset/MNIST/images/{index:05d}_{label}.jpg", image)
    index += 1